# <center>**Taller 06: Bases de Datos Vectoriales**</center>

### **Integrantes:**
- Calahorrano David
- Códorva Carlos
- Zambrano Ándres

### **Objetivo:**
En este ejercicio implementarás un sistema de recuperación que utilice representaciones vectoriales de texto y compararás este enfoque con los modelos TF-IDF y BM25.

### **Requisitos:**
1. Librerías Python necesarias: sentence-transformers, FAISS, ChromaDB.
2. Instalar Elasticsearch en un contenedor utilizando Docker.
3. Dataset Wikipedia Movie Plots, disponible en Kaggle, y utilizado en el taller 05.

---

### **Conceptos Clave**
#### **TF-IDF**
- **Definición:** TF-IDF (Term Frequency - Inverse Document Frequency) es una métrica que mide la relevancia de un término dentro de un documento respecto a un corpus. Combina dos factores:
- **Frecuencia del término (TF):** Qué tan frecuente es una palabra en un documento.
- **Frecuencia inversa en el corpus (IDF):** Penaliza palabras comunes en el corpus.

#### **BM25**
- **Definición:** Una extensión de TF-IDF que ajusta la relevancia basada en:
  - La longitud del documento (normalización).
  - La saturación de la frecuencia del término (evita que palabras muy repetidas dominen el puntaje).
- **Ventajas:** Proporciona una búsqueda de texto más precisa y es el método predeterminado en Elasticsearch.

#### **Embeddings**
- **Definición:** Representaciones vectoriales de texto que capturan relaciones semánticas entre palabras o frases. Los embeddings transforman palabras o documentos en vectores en un espacio multidimensional donde textos similares están cerca entre sí.
- **Herramienta:** Librería sentence-transformers, que ofrece modelos preentrenados como all-MiniLM-L6-v2 para generar embeddings de alta calidad.

#### **Bases de Datos Vectoriales (FAISS y ChromaDB)**
- **FAISS:** Biblioteca especializada en búsquedas rápidas en grandes colecciones de vectores. Diseñada para escalabilidad y velocidad.
- **ChromaDB:** Base de datos orientada a embeddings que ofrece funcionalidades avanzadas para almacenar y consultar vectores de manera eficiente.

#### **Comparación de Estrategias de Recuperación**
- **TF-IDF vs. BM25 vs. Embeddings:**
  - **TF-IDF:** Mide la relevancia de palabras clave basándose en su frecuencia relativa.
  - **BM25:** Mejora TF-IDF ajustando por la longitud del documento y saturación de términos.
  - **Embeddings:** Capturan relaciones semánticas, permitiendo recuperar documentos relevantes, aunque las palabras exactas no coincidan.

#### **Preparación del Dataset: Wikipedia Movie Plots**
1. Usa el dataset del Taller 05.

---

### **Instrucciones**
#### **Parte 1: Recuperación con TF-IDF**
1. Cargar los datos en Python
2. Configurar TF-IDF:
   - Usa la librería scikit-learn para calcular los puntajes TF-IDF de los plots.
3. Realizar consultas:
   - Escribe una función que calcule la similitud entre una consulta y los documentos usando la matriz TF-IDF.
4. Evaluar los resultados:
   - Registra los documentos recuperados y analiza su relevancia.

#### **Parte 2: Recuperación con BM25**
1. Configurar Elasticsearch:
   - Reutiliza el índice creado en el Ejercicio 1 para realizar consultas basadas en BM25.
2. Realizar consultas:
   - Usa palabras clave como "dinosaurs" o "cyborg" para encontrar documentos relevantes.
3. Evaluar los resultados:
   - Compara los documentos recuperados con los obtenidos usando TF-IDF.

#### **Parte 3: Recuperación con FAISS**
1. Configurar FAISS:
   - Crea un índice en FAISS y agrega los embeddings generados previamente.
2. Realizar consultas:
 - Convierte una consulta en texto (e.g., "A park with cloned dinosaurs") en un vector usando el mismo modelo de embeddings.
 - Busca los vectores más cercanos en FAISS.
3. Evaluar los resultados:
- Compara los documentos recuperados por FAISS con los obtenidos usando TF-IDF y BM25.

#### **Parte 4: Recuperación con ChromaDB**
1. Configurar ChromaDB:
- Inicia una base de datos de ChromaDB y define el esquema con los campos Title, Plot y Embedding.
2. Insertar documentos y embeddings:
- Agrega los documentos y sus embeddings generados previamente a ChromaDB, junto con los metadatos correspondientes (e.g., título y trama).
3. Realizar consultas:
- Convierte una consulta de texto en un embedding.
- Busca los vectores más cercanos en ChromaDB y recupera los documentos relacionados.
4. Evaluar los resultados:
- Compara los documentos recuperados por ChromaDB con los de FAISS, TF-IDF y BM25.

#### **Parte 5: Comparación de Resultados**
1. Relevancia:
   - Analiza cuál de las estrategias (TF-IDF, BM25, FAISS o ChromaDB) recupera documentos más relevantes para diferentes consultas.
2. Ventajas y limitaciones:
   - Reflexiona sobre los puntos fuertes y débiles de cada enfoque.